# phase 1


## Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib

## Cleaning and Preprcessing

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/PROJECTS/ZOMATODATAANALYSIS/DATASETS/Copy of zomato.csv")

In [ ]:
data = df.drop(['url', 'address', 'phone', 'dish_liked', 'reviews_list', 'menu_item', 'location', 'name'], axis=1)

In [ ]:
data = data.rename(columns={'approx_cost(for two people)': 'cost',
               'listed_in(type)': 'type',
               'listed_in(city)': 'city'})

In [ ]:
data.dropna(how='any', inplace=True)    #dropping nul values from the data

In [ ]:
data.drop_duplicates(inplace=True)    #dropping the duplicates

In [ ]:
data['online_order'] = [0 if x == 'Yes' else 1 for x in data['online_order']]
data['book_table'] = [0 if x == 'Yes' else 1 for x in data['book_table']]

In [ ]:
data = data.drop(['city'], axis=1)

In [ ]:
data = data[data['rate'] != 'NEW']
data = data[data['rate'] != '-']

In [ ]:
data['cost'] = data['cost'].replace(',', '', regex=True)

In [ ]:
data['cost'] = data['cost'].astype(float)

In [ ]:
data['rate'] = data['rate'].replace('/5', '', regex=True)

In [ ]:
data['rate'] = data['rate'].astype(float)

In [ ]:
data.to_csv('preprocessed.csv', index=False)

checkpoint


In [10]:
df = pd.read_csv("Datasets\preprocessed.csv")

In [2]:
def expand_dataframe(df, column):
    df_expanded = df.copy()
    df_expanded[column] = df_expanded[column].str.split(', ')
    df_expanded = df_expanded.explode(column)
    return df_expanded

In [11]:
df_expanded = expand_dataframe(df, 'rest_type')
df_expanded = expand_dataframe(df_expanded, 'cuisines')

In [12]:
type_mean = df_expanded.groupby('type')['rate'].mean()
rest_type_mean = df_expanded.groupby('rest_type')['rate'].mean()
cuisines_mean = df_expanded.groupby('cuisines')['rate'].mean()

In [13]:
df_encoded = df_expanded.copy()
df_encoded['type'] = df_expanded['type'].map(type_mean)
df_encoded['rest_type'] = df_expanded['rest_type'].map(rest_type_mean)
df_encoded['cuisines'] = df_expanded['cuisines'].map(cuisines_mean)

In [14]:
df_encoded = df_encoded.fillna(df_encoded.mean())

In [24]:
df_encoded.to_csv('preprocessed_encoded.csv', index=False)

In [27]:
df

,online_order,book_table,rate,votes,rest_type,cuisines,cost,type
0,0,0,4.1,775,Casual Dining,"North Indian, Mughlai, Chinese",800.0,Buffet
1,0,1,4.1,787,Casual Dining,"Chinese, North Indian, Thai",800.0,Buffet
2,0,1,3.8,918,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,Buffet
3,1,1,3.7,88,Quick Bites,"South Indian, North Indian",300.0,Buffet
4,1,1,3.8,166,Casual Dining,"North Indian, Rajasthani",600.0,Buffet
...,...,...,...,...,...,...,...,...
41163,1,1,3.7,34,"Casual Dining, Bar","North Indian, Continental",800.0,Pubs and bars
41164,1,1,2.5,81,"Casual Dining, Bar","Andhra, South Indian, Chinese, North Indian",800.0,Pubs and bars
41165,1,1,3.6,27,Bar,Continental,1500.0,Pubs and bars
41166,1,0,4.3,236,Bar,Finger Food,2500.0,Pubs and bars


## Train Test Split

In [15]:
X = df_encoded.drop('rate', axis=1)
y = df_encoded['rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model 2: Random Forest Regressor

In [16]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [17]:
y_pred = rf_regressor.predict(X_test)

In [18]:
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

r2_score = rf_regressor.score(X_test, y_test)

print(f'Root Mean Squared Error: {rmse}')
print(f'R^2 Score: {r2_score}')

Root Mean Squared Error: 0.13247801414617044
R^2 Score: 0.9165887272781876


### Dump :)

In [19]:
joblib.dump(rf_regressor, 'random_forest_model_expanded.pkl')

['random_forest_model_expanded.pkl']

## proto 1


In [20]:
import pandas as pd
import joblib

# Load the saved model
rf_regressor = joblib.load('random_forest_model_expanded.pkl')

# Function to expand the dataframe for multi-category cells
def expand_dataframe(df, column):
    df_expanded = df.copy()
    df_expanded[column] = df_expanded[column].str.split(', ')
    df_expanded = df_expanded.explode(column)
    return df_expanded

# Function to preprocess input data and predict the rating
def predict_restaurant_rating(online_order, book_table, votes, rest_type, cuisines, cost, type):
    # Create a dataframe for the input data
    input_data = pd.DataFrame({
        'online_order': [online_order],
        'book_table': [book_table],
        'votes': [votes],
        'rest_type': [rest_type],
        'cuisines': [cuisines],
        'cost': [cost],
        'type': [type]
    })

    # Expand the dataframe for 'rest_type' and 'cuisines'
    input_data = expand_dataframe(input_data, 'rest_type')
    input_data = expand_dataframe(input_data, 'cuisines')

    # Map the means to the respective categories
    input_data['type'] = input_data['type'].map(type_mean)
    input_data['rest_type'] = input_data['rest_type'].map(rest_type_mean)
    input_data['cuisines'] = input_data['cuisines'].map(cuisines_mean)

    # Fill NaN values resulting from unmapped categories with the overall mean
    input_data = input_data.fillna(df_encoded.mean())

    # Predict the rating using the trained model and take the mean of predictions
    predicted_rating = rf_regressor.predict(input_data).mean()

    return predicted_rating

In [21]:
def get_user_input_and_predict():
    online_order = int(input("Does the restaurant accept online orders? (1 for Yes, 0 for No): "))
    book_table = int(input("Can the restaurant book tables? (1 for Yes, 0 for No): "))
    votes = int(input("Enter the number of votes: "))
    rest_type = input("Enter the restaurant types (comma separated if multiple): ")
    cuisines = input("Enter the cuisines (comma separated if multiple): ")
    cost = float(input("Enter the average cost for two people: "))
    type = input("Enter the restaurant type (Buffet, Casual Dining, etc.): ")

    predicted_rating = predict_restaurant_rating(
        online_order=online_order,
        book_table=book_table,
        votes=votes,
        rest_type=rest_type,
        cuisines=cuisines,
        cost=cost,
        type=type
    )

    print(f'Predicted Rating: {predicted_rating:.2f}')

In [23]:
# Run the function to get user input and predict the rating
get_user_input_and_predict()

KeyboardInterrupt: Interrupted by user